###Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [14]:
def get_currency_rate_and_name(currency):
    if 'Name' not in currency or 'Value' not in currency:
        raise RuntimeError('Malformed currency response info')
    return {'name': currency['Name'], 'rate': currency['Value']}


def get_currency_with_max_rate():
    """
    :return: currency with max rate
    :rtype: dict
    """
    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

    if response.status_code != 200:
        raise RuntimeError('Bad HTTP response code from currency server')

    json = response.json()

    max_currency = None

    if 'Valute' in json:
        for key, currency in json['Valute'].items():
            current_currency = get_currency_rate_and_name(currency)

            if max_currency is None or max_currency['rate'] < current_currency['rate']:
                max_currency = current_currency
    else:
        raise RuntimeError('Response format is incorrect, no Valute entry')

    return max_currency



##Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [10]:
import requests

In [23]:
class Rate:
    def __init__(self, format='value', diff=False):
        self.format = format
        self.__diff = diff

    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']

    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            if self.format == 'value':
                return response[currency]['Value']

            if self.format == 'name':
                return response[currency]['Name']

        return 'Error'

    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.__get_format_response('EUR')

    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.__get_format_response('USD')

    def __get_currency_diff(self, currency_code):
        resp = self.exchange_rates()

        if currency_code in resp:
            return resp[currency_code]['Value'] - resp[currency_code]['Previous']

    def __get_format_response(self, currency_code):
        return self.make_format(currency_code) if not self.__diff else self.__get_currency_diff(currency_code)

In [29]:
r = Rate()

In [30]:
r.exchange_rates()


{'AUD': {'ID': 'R01010',
  'NumCode': '036',
  'CharCode': 'AUD',
  'Nominal': 1,
  'Name': 'Австралийский доллар',
  'Value': 47.1035,
  'Previous': 46.868},
 'AZN': {'ID': 'R01020A',
  'NumCode': '944',
  'CharCode': 'AZN',
  'Nominal': 1,
  'Name': 'Азербайджанский манат',
  'Value': 45.9687,
  'Previous': 46.593},
 'GBP': {'ID': 'R01035',
  'NumCode': '826',
  'CharCode': 'GBP',
  'Nominal': 1,
  'Name': 'Фунт стерлингов Соединенного королевства',
  'Value': 92.2778,
  'Previous': 91.82},
 'AMD': {'ID': 'R01060',
  'NumCode': '051',
  'CharCode': 'AMD',
  'Nominal': 100,
  'Name': 'Армянских драмов',
  'Value': 15.7555,
  'Previous': 15.9695},
 'BYN': {'ID': 'R01090B',
  'NumCode': '933',
  'CharCode': 'BYN',
  'Nominal': 1,
  'Name': 'Белорусский рубль',
  'Value': 30.1674,
  'Previous': 30.1608},
 'BGN': {'ID': 'R01100',
  'NumCode': '975',
  'CharCode': 'BGN',
  'Nominal': 1,
  'Name': 'Болгарский лев',
  'Value': 43.0246,
  'Previous': 43.6451},
 'BRL': {'ID': 'R01115',
  'NumC

In [31]:
r.usd()

77.7928

###Задание 3
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены

In [56]:
class Employee:
    def __init__(self, name, seniority, awards):
        self.name = name
        self.seniority = seniority 
        self.awards = awards * 2      
        self.grade = 1          
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [70]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority, awards)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.awards +=2
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if ((self.awards * 2) + self.seniority) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [71]:
elena = Designer('Елена', 0, 2)

In [72]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
